[Node 15: SWIG](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node15.html)

Navigation:

**Next:** [Threads and Multi–Processing](node16.ipynb) **Up:** [Threads and Multi–Processing](node16.ipynb) **Previous:** [Threads and Multi–Processing](node16.ipynb)

## SWIG
To connect a C or C++ library to Python, we use the SWIG interface generator (SWIG = Simplified Wrapper and Interface Generator). This module is not part of the standard Python installation and must be installed separately, e.g. with ``sudo apt-get install swig`` on Ubuntu systems. (It is not limited to Python, but can also be used as an interface to other languages.)
 

### C Classes
We use the example from the [Tutorial](http://www.swig.org/tutorial.html).
The file ``example.c`` should be included in Python:

```C++
/* File : example.c */
 
 #include <time.h>
 double My_variable = 3.0;
 
 int fact(int n) {
     if (n <= 1) return 1;
     else return n*fact(n-1);
 }
 
 int my_mod(int x, int y) {
     return (x%y);
 }
        
 char *get_time()
 {
     time_t ltime;
     time(&ltime);
     return ctime(&ltime);
 }
```

To wrap the functions ``fact, my_mod, get_time`` and the variable ``My_variable`` from ``example.c``, we need the following interface file ``example.i`` :

```C++
/* example.i */
 %module example
 %{
 /* Put header files here or function declarations like below */
 extern double My_variable;
 extern int fact(int n);
 extern int my_mod(int x, int y);
 extern char *get_time();
 %}
 
 // list functions swig should generate wrappers for
 extern double My_variable;
 extern int fact(int n);
 extern int my_mod(int x, int y);
 extern char *get_time();
```

The `%module` directive defines the name of the module to be generated and contains other code (headers and declarations) that is copied into the SWIG-generated wrapper.
The [zweite Teil](https://stackoverflow.com/questions/32208350/swig-why-we-need-to-declare-functions-twice) contains the definitions of those functions for which a wrapper is to be created.

SWIG can use this interface file to generate 'glue-code' for some languages ​​(e.g. Perl, Java). For Python, SWIG is invoked as follows, creating the files ``example.py`` and ``example_wrap.c.`` :

**( change to sub-dir ./source/swig )**

```bash
swig -python example.i
```
Now ``example_wrap.c.`` needs to be compiled into a shared library called ``_example.so``:
```bash
gcc -fPIC -c example.c example_wrap.c -I/usr/include/python3.5/
ld -shared example.o example_wrap.o -o _example.so
```

Within Python, the new module ``example`` and the corresponding routines can now be called as follows:

In [ ]:
from source.swig import example
example.fact(5)

In [ ]:
dir(example)

In [ ]:
example.my_mod(7,3)

In [ ]:
example.get_time()

In [ ]:
example.cvar.My_variable

Everything in the ``%{ ... % }`` block is pasted unmodified into the auto-generated ``example_wrap.c``.
 

### Compile automatically
The following Python program ``example_setup.py`` (uses [``distutils``](https://docs.python.org/3/library/distutils.html)) can be used to compile the entire program code at once and combine it into a shared library:
```python
#!/usr/bin/env python
from distutils.core import setup, Extension
example_module = Extension('_example', sources = ['example.c','example_wrap.c'])
setup ( name = 'example',
        version = '0.1',
        author = 'John Doe',
        description = "An example program",
        ext_modules = [example_module],
        py_modules = ['example']
)
```

Then create the module ``example`` with the following line:
```bash
python example_setup.py build_ext --inplace
```
(`swig` must have been called beforehand.)

### C++ classes

C++ classes are included just like C classes. A class ``Person`` is declared in header file [person.h](source/swig/person.h). The member functions are declared in the [person.h](source/swig/person.h) file. A test program [person.h](source/swig/person.h) demonstrates the use of the ``Person`` class. This C++ program can be compiled and started as follows:
```bash
g++ person.cxx person_test.cxx -o person_test
./person_test
```

In the SWIG wrapper [person.i](source/swig/person.i) the whole class ``Person`` is defined unchanged. The [person.i](source/swig/person.i) file is used to automatically compile the code. The SWIG glue code is generated with the following line:

```bash
swig -python -c++ person.i
```

The entire ``person`` module is created with the following command:
```bash
python3 person_setup.py build_ext --inplace
```
(You may need to rename the `_person*.so` to `_person.so`.

Within Python, the ``Person`` module can be used like this:

In [ ]:
from source.swig.person import Person
p=Person('Max Student')
p.get_name()

In [ ]:
p.set_name('Maria Studentin')
p.get_name()

In [ ]:
p2=Person(p)
p2.get_name()

In [ ]:
# quit() causes to the kernel to stop.
# We use it to see that the destructor is called.
# (Check the output in the console where you started the notebook server.)
quit()